### Tratamento do texto do campo DECISAO do dataset de Acórdãos

In [1]:
import re
import tools
import string
import pyperclip

import pandas as pd
# import texthero as hero
# import pre_processing as pp

from utils import highlight_regex_matches as hl

from datetime import datetime

In [ ]:
# Configuração para mostrar todas as linhas de um dataframe
pd.set_option('display.max_rows', None)

In [ ]:
df_acordaos = tools.read_file('dados/df_acordaos.csv')
df_acordaos.info()

In [2]:
df_rotulados = tools.read_file('dados/df_rotulados.csv')
# df_rotulados.info()

Duration: 0:00:00.275671


In [311]:
acordao = r'([Aa]c[óo]rd[ãa]os?)'
alinea = r'(al[ií]neas?)'
ano = r'([0-9]{2,4})?'
artigo = r'(artigos?|\barts?\.?)'
barra = r'(\/)'
camara = r'[cC][aâ]mara'
caput = r'([cC]aput)'
cef = r'(Caixa Econ[oô]mica Federal)'
combinado_com = r'(c\/c)'
constituicao = r'(\b[cC]\.?[fF]\.?\b|[cC]onstitui[cç][aã]o [fF]ederal|[cC]arta [mM]agna|[cC]arta [mM]agna (do)? ([bB]rasil|[bB]rasileira)\b)'
contrato = r'(\b[cC]ontrato\b)'
da = r'(da)?'
divisor = r'([\/|-])'
data_full = r'([0-9]{1,2}[\°|\º]?)'+divisor+'([0-9]{2})'+divisor+'([0-9]{2,4})'# r'([0-9]{2})'+divisor+'([0-9]{2})'+divisor+'([0-9]{2,4})'
de = r'(de)?'
decreto = r'([dD]ecreto)'
do = r'(do)?'
e = r'(e)?'
hifen = r'(-)*'
lei = r'([Ll][Ee][Ii])'
decreto_lei = decreto + hifen + lei
espaco = r'(\s)*'
inciso = r'(incisos?|\bincs?.?\b)'
instrucao_normativa = r'(\bInstru[cç][aã]o Normativa\b)'
ldo = r'([lL][dD][oO]|'+ lei + r'[dD]iretrizes [oO]r[cç]ament[aá]rias)'
lei_complementar = lei + r'([cC]omplementar)'
lei_licitacacao = lei + espaco + r'de' + espaco + r'([lL]icita[cç]([aã]o|[oõ]es))'
numero = r'(\bn[\°|\º])?'
numerais = r'([\d.]{1,})'
ordinais = r'[\°|\º|\ª]?'
paragrafo = r'(§ §|§§?|[pP]aragr[aá]fos?)'
plenario = r'([pP]len[aá]rio)'
ponto = r'\.?'
portaria = r'(\b[pP]ortaria\b)'
regimento_interno = r'([Rr]egimento [Ii]nterno)'
romano = r'([MDCLXVI]+\b)'
r'\b([MDCLXVI]+)\b ?,? ?e? ?'
sumula = r'(\b[sS][uú]mula\b)'
tcu = r'(\b[tT][cC][uU]|[tT]ribunal de [cC]ontas da [uU]nião\b)'
tcu_opt = r'(tcu)*'
todos = r'(todos)'
virgula = r'(,)?'
underline = r'_'

In [314]:
# Definição dos padrões a serem buscados no texto e de como eles serão substituídos
pa_lei_ano = re.compile(lei+espaco+numero+espaco+numerais+ordinais+divisor+ano, re.I)
re_lei_ano = r'lei_\5_\7'

pa_lei_datafull = re.compile(lei+espaco+numero+espaco+numerais+ordinais+virgula+espaco+de+espaco+data_full, re.I)
re_lei_datafull = r'lei_\5_\14'

pa_decreto_ano = re.compile(decreto+espaco+numerais+ordinais+divisor+ano, re.I)
re_decreto_ano = r'decreto_\3_\6'

pa_decreto_datafull = re.compile(decreto+espaco+numero+espaco+numerais+ordinais+virgula+espaco+de+espaco+data_full, re.I)
re_decreto_datafull = r'decreto_\5_15'

pa_artigo = re.compile(artigo+espaco+numerais+ordinais)
re_artigo = r'decreto_\3'

pa_paragrafo = re.compile(paragrafo+espaco+numerais+ordinais)
re_paragrafo = r'paragrafo\3'

pa_inciso = re.compile(inciso+espaco+romano, re.I)
re_inciso = r'inciso_\3'

pa_cef = re.compile(cef, re.I)
re_cef = r'cef_'

pa_constituicao = re.compile(constituicao, re.I)
re_constituicao = r'c_f'

# pa_instrucao_normativa
# re_instrucao_normativa

pa_tcu = re.compile(tcu, re.I)
re_tcu = r'tcu'

pa_ritcu = re.compile(regimento_interno+espaco+do+espaco+tcu, re.I)
re_ritcu = r'ri_tcu'

pa_acpl = re.compile(acordao+espaco+numerais+ordinais+divisor+ano+espaco+hifen+espaco+tcu_opt+espaco+hifen+espaco+plenario, re.I)
re_acpl = r'acordao_\3_\5_PL'

pa_ac1c = re.compile(acordao+espaco+numerais+ordinais+divisor+ano+espaco+hifen+espaco+tcu_opt+espaco+hifen+espaco+'1'+ordinais+espaco+camara, re.I)
re_ac1c = r'acordao_\3_\5_1C'

pa_ac2c = re.compile(acordao+espaco+numerais+ordinais+divisor+ano+espaco+hifen+espaco+tcu_opt+espaco+hifen+espaco+'2'+ordinais+espaco+camara, re.I)
re_ac2c = r'acordao_\3_\5_2C'

pa_ldo = re.compile(ldo, re.I)
re_ldo = r'ldo'

In [4]:
def padronizacao(row, pattern, replace):
    compiled = re.compile(pattern)

    return re.sub(compiled, replace, row.DECISAO)
    


In [312]:
# texto = 'Acórdão 496/2004 - TCU - 2ª Câmara. Acórdão 615/2004 - 2ª Câmara'
texto = df_rotulados.iloc[77].DECISAO

pa = re.compile(ldo, re.I)
replace = r'ldo'

# Para efeito de teste deve-se compilar a regex com a flat 're.I' para usar o highlight_regex_matches
pa2 = re.compile(r'ldo')
hl(pa2, pa.sub(replace, texto))
# hl(re.compile(replace), pa.sub(replace, texto))

VISTOS, relatados e discutidos estes autos de auditoria de conformidade realizada em obras de penitenciárias sob a responsabilidade do Departamento Penitenciário Nacional (Depen/MJ), por determinação do Acórdão 450/2006-TCU-Plenário, objetivando aferir se os custos das obras estão de acordo com os preços de mercado e se as determinações proferidas por este Tribunal por meio do Acórdão 496/2004-TCU-2ª Câmara estão sendo cumpridas,ACORDAM os Ministros do Tribunal de Contas da União, reunidos em sessão do Plenário, ante as razões expostas pelo Relator, em:9.1. determinar ao Ministério da Justiça, com fulcro no art. 43, inciso I, da Lei 8.443/92 c/c o art. 250, inciso II, do Regimento Interno do TCU, que:9.1.1. promova o ajuste dos Contratos 01/2003, de 22/08/2003, e 03/2004, de 27/12/2004, firmados com a Caixa Econômica Federal, caso ainda vigentes,com a finalidade de que sejam elaboradas planilhas orçamentárias detalhadas de serviços, contemplando os respectivos quantitativos e composiçõ

In [203]:
pyperclip.copy(df_rotulados.iloc[77].DECISAO)

In [ ]:
pyperclip.copy(decreto+espaco+numerais+divisor+ano)

In [ ]:
texto = 'teste de texto decreto n° 2515, de 11/05/1988. Acórdão 496/2004 - TCU - 2ª Câmara com numerais art. 7º, § 2º, inciso II, III e IV e art. 40, § 2º, inciso II, todos da Lei nº 8.666, de 12/11/1993 verificando lei n° 4835/1952'

pa_artigo_paragrafo_lei = r'('+artigo+espaco+numerais+virgula+espaco+paragrafo+espaco+numerais+espaco+virgula+espaco+inciso+espaco+romano+virgula+\
espaco+e+espaco+r')+'+todos+espaco+da+espaco+lei+espaco+numero+espaco+numerais+divisor+ano

pa_artigo_inciso_lei = artigo+espaco+numerais+virgula+espaco+inciso+espaco+romano+virgula+\
                 espaco+da+espaco+lei+espaco+numerais+divisor+ano
re_lei_artigo_inciso = r'LEI_\15_\17_ARTIGO_\3_INCISO_\8'

pa_lei_ano = re.compile(lei+espaco+numero+espaco+numerais+divisor+ano)
pa_lei_datafull = re.compile(lei+espaco+numero+espaco+numerais+virgula+espaco+de+espaco+data_full)

pa_decreto_ano = decreto+espaco+numero+espaco+numerais+ano_divisor+ano
pa_decreto_datafull = decreto+espaco+numero+espaco+numerais+virgula+espaco+de+espaco+data_full

romano = r'\b[MDCLXVI]+ ?,? ?e? ?'
pa_artigo_paragrafo_inciso = artigo+espaco+numerais+virgula+espaco+paragrafo+espaco+numerais+espaco+virgula+espaco+inciso+espaco+romano\
+virgula+espaco

re_lei = r'LEI_\5_\7'

pattern = re.compile(pa_artigo_paragrafo_inciso)

hl(pattern, texto)

In [ ]:
# Aplicação do regex no texto
df_rotulados.DECISAO = df_rotulados.apply(lambda x: padronizacao(x, pa_lei_ano, re_lei_ano), axis = 1)

In [317]:
hl(re.compile(lei+underline+numerais+underline+ano), df_rotulados.apply(lambda x: padronizacao(x, pa_lei_ano, re_lei_ano), axis = 1)[77])

VISTOS, relatados e discutidos estes autos de auditoria de conformidade realizada em obras de penitenciárias sob a responsabilidade do Departamento Penitenciário Nacional (Depen/MJ), por determinação do Acórdão 450/2006-TCU-Plenário, objetivando aferir se os custos das obras estão de acordo com os preços de mercado e se as determinações proferidas por este Tribunal por meio do Acórdão 496/2004-TCU-2ª Câmara estão sendo cumpridas,ACORDAM os Ministros do Tribunal de Contas da União, reunidos em sessão do Plenário, ante as razões expostas pelo Relator, em:9.1. determinar ao Ministério da Justiça, com fulcro no art. 43, inciso I, da lei_8.443_92 c/c o art. 250, inciso II, do Regimento Interno do TCU, que:9.1.1. promova o ajuste dos Contratos 01/2003, de 22/08/2003, e 03/2004, de 27/12/2004, firmados com a Caixa Econômica Federal, caso ainda vigentes,com a finalidade de que sejam elaboradas planilhas orçamentárias detalhadas de serviços, contemplando os respectivos quantitativos e composiçõ

In [318]:
n_lei = r'(?P<lei>[\d.]{1,}[\°|\º]?)'
n_artigo = r'(?P<artigo>[\d.]{1,}[\°|\º]?)'
pa_artigo_inciso_lei = artigo+espaco+n_artigo+virgula+espaco+inciso+espaco+romano+virgula+\
                     espaco+da+espaco+lei+espaco+n_lei+barra+ano
# Usando backreferences para fazer as substituições
re_lei_artigo_inciso = r'LEI_\15_\17_ARTIGO_\3_INCISO_\8'
pattern = re.compile(pa_artigo_inciso_lei)
teste = re.sub(pattern, re_lei_artigo_inciso, df_rotulados.iloc[77].DECISAO)

# teste = re.sub(pattern, re_lei_artigo_inciso, df_rotulados.iloc[245].DECISAO)
pattern2 = re.compile(lei+underline+numerais+underline+ano+underline+r'ARTIGO'+underline+numerais+underline+r'INCISO'+underline+romano)

hl(pattern2, teste)

VISTOS, relatados e discutidos estes autos de auditoria de conformidade realizada em obras de penitenciárias sob a responsabilidade do Departamento Penitenciário Nacional (Depen/MJ), por determinação do Acórdão 450/2006-TCU-Plenário, objetivando aferir se os custos das obras estão de acordo com os preços de mercado e se as determinações proferidas por este Tribunal por meio do Acórdão 496/2004-TCU-2ª Câmara estão sendo cumpridas,ACORDAM os Ministros do Tribunal de Contas da União, reunidos em sessão do Plenário, ante as razões expostas pelo Relator, em:9.1. determinar ao Ministério da Justiça, com fulcro no LEI_8.443_92_ARTIGO_43_INCISO_I c/c o art. 250, inciso II, do Regimento Interno do TCU, que:9.1.1. promova o ajuste dos Contratos 01/2003, de 22/08/2003, e 03/2004, de 27/12/2004, firmados com a Caixa Econômica Federal, caso ainda vigentes,com a finalidade de que sejam elaboradas planilhas orçamentárias detalhadas de serviços, contemplando os respectivos quantitativos e composições 

In [ ]:
from texthero import preprocessing

In [ ]:
custom_pipeline = [preprocessing.lowercase,
                  preprocessing.remove_diacritics,
                  preprocessing.remove_whitespace]

In [ ]:
start_time = datetime.now()
df_rotulados['EDITADO'] = df_rotulados.DECISAO.pipe(hero.clean)
print('Duration: {}'.format(datetime.now() - start_time))